In [21]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")


# OpenAI version

In [47]:
prompt = ["Write a hello world program in Python", "What is the fourth prime number?"]
response = openai.Completion.create(
    model="text-davinci-002",
    prompt=prompt,
    temperature=1.0,
    max_tokens=64,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    logprobs=1,
    echo=True,
    n=2,
    stop=["\"\"\""]
)

In [48]:
[response['choices'][i]['text'] for i in range(len(response['choices']))]

['Write a hello world program in Python\n\nprint("Hello world!")',
 'Write a hello world program in Python\n\nprint("Hello, world!")',
 'What is the fourth prime number?\n\n7',
 'What is the fourth prime number?\n\n7']

In [49]:
log_probs = [response['choices'][i]['logprobs']['token_logprobs'][1:]
            for i in range(len(response['choices']))]
tokens = [response['choices'][i]['logprobs']['tokens'][1:]
        for i in range(len(response['choices']))]
offsets = [response['choices'][i]['logprobs']['text_offset'][1:]
        for i in range(len(response['choices']))]

# Subtract 1 from the offsets to account for the newline
for i in range(len(offsets)):
    offsets[i] = [offset - 1 for offset in offsets[i]]

# if log_prob_range is not None:
#     # First, we need to find the indices of the tokens in the log probs
#     # that correspond to the tokens in the log_prob_range
#     for i in range(len(log_probs)):
#         lower_index, upper_index = self.get_token_indices(
#             offsets[i], log_prob_range[i])
#         log_probs[i] = log_probs[i][lower_index:upper_index]
#         tokens[i] = tokens[i][lower_index:upper_index]

In [50]:
tokens[0]

[' a',
 ' hello',
 ' world',
 ' program',
 ' in',
 ' Python',
 '\n',
 '\n',
 'print',
 '("',
 'Hello',
 ' world',
 '!"',
 ')']

In [68]:
log_probs[0]

[-0.3710930645465851,
 -3.699204444885254,
 -1.6033620834350586,
 -0.07942259311676025,
 -0.0023649362847208977,
 -0.5106741189956665,
 -5.155109405517578,
 -1.1185113191604614,
 -3.548737049102783,
 -1.20339035987854,
 -0.037674229592084885,
 -0.8867388367652893,
 -1.3799138069152832,
 -0.006083305925130844,
 -0.0009765623253770173,
 -1.0728830375228426e-06,
 -0.0003610197745729238,
 -0.0073336209170520306,
 -2.0263919830322266,
 -0.06149892508983612,
 -0.2051912397146225,
 -1.7620773315429688,
 -0.0040281834080815315,
 -0.8398252725601196,
 -0.03858637437224388,
 -0.026370810344815254,
 -3.7345595359802246,
 -4.294748783111572,
 -1.0190117359161377]

In [69]:
offsets[0]

[4, 6, 12, 18, 26, 29, 36, 37, 38, 43, 45, 50, 56, 58]

In [52]:
len(tokens[0]), len(log_probs[0])

(14, 14)

# HuggingFace version

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import numpy as np

dev = 'cuda'

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl", device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer

T5TokenizerFast(name_or_path='google/flan-t5-xl', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', 

In [6]:
def do_predict(input_texts, model, tokenizer, n=1, logprobs = False):
    """
    Takes in a list of text inputs and generates output text for each input using the model and tokenizer.

    Args:
        input_texts (List[str]): List of text inputs to be used as input to the model.
        model (transformers.PreTrainedModel): Model to be used for prediction.
        tokenizer (transformers.PreTrainedTokenizer): Tokenizer to be used for prediction.
        n (int, optional): Number of predictions to generate for each input. Defaults to 1.

    Returns:
        List[str]: List of generated text.
    """
    # Encode the input texts
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True).to(model.device)

    # Generate output with the model
    outputs = model.generate(
        **inputs,
        max_length=30,
        num_return_sequences=n,
        do_sample=True,
        return_dict_in_generate=True,
        output_scores=True)

    transition_scores = model.compute_transition_scores(
        outputs.sequences, outputs.scores, normalize_logits=True
    )

    # ref: https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.compute_transition_scores
    # input_length is the length of the input prompt for decoder-only models, like the GPT family, and 1 for
    # encoder-decoder models, like BART or T5.
    input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
    generated_tokens = outputs.sequences[:, input_length:]

    if logprobs:

        filtered_tokens = []
        filtered_scores = []

        # remove padding and eos tokens, which have a score of -inf
        for seq_tokens, seq_scores in zip(generated_tokens, transition_scores):
            unpadded_tokens = [token for token in seq_tokens if token not in (tokenizer.pad_token_id, tokenizer.eos_token_id)]
            unpadded_scores = [score for token, score in zip(seq_tokens, seq_scores) if token not in (tokenizer.pad_token_id, tokenizer.eos_token_id)]
            
            filtered_tokens.append(unpadded_tokens)
            filtered_scores.append(unpadded_scores)

        token_seq = [[tokenizer.decode(token_id, clean_up_tokenization_spaces=True, skip_special_tokens=False) for token_id in tokens] for tokens in filtered_tokens]
        
        return filtered_scores, token_seq    
        # token_seq = [[tokenizer.decode(token_id, clean_up_tokenization_spaces = True, skip_special_tokens=False) for token_id in tokens] for tokens in generated_tokens]
        # log_probs = [transition_scores[i].tolist() for i in range(transition_scores.shape[0])]
        # return log_probs, token_seq

    else:
        # Convert tokens to text
        generated_texts = tokenizer.batch_decode(generated_tokens, clean_up_tokenization_spaces = True, skip_special_tokens=True)

        return generated_texts

In [10]:
log_probs, tokens = do_predict(["Write a hello world program in Python", "What is the fourth prime number?"], model, tokenizer, n=3, logprobs = True)
for tok, score in zip(tokens[0], log_probs[0]):
    # | token | token string | logits | probability
    print(f"| {tok:8s} | {score.cpu():.3f} | {np.exp(score.cpu()):.2%}")


| import   | -2.755 | 6.36%
|          | -0.253 | 77.62%
| i        | -3.746 | 2.36%
| o        | -0.019 | 98.11%
| ,        | -0.807 | 44.63%
|          | -0.227 | 79.67%
| o        | -0.524 | 59.24%
| s        | -0.002 | 99.80%
| import   | -2.029 | 13.14%
|          | -0.222 | 80.09%
| s        | -0.227 | 79.66%
| y        | -0.013 | 98.75%
| s        | -0.000 | 100.00%
| import   | -2.124 | 11.96%
|          | -1.383 | 25.07%
| re       | -1.154 | 31.54%
| ,        | -2.567 | 7.67%
|          | -0.388 | 67.83%
| s        | -1.048 | 35.08%
| y        | -0.744 | 47.53%
| s        | -0.002 | 99.82%
| .        | -0.246 | 78.20%
| set      | -0.434 | 64.80%
| re       | -0.000 | 99.97%
| cur      | -0.000 | 100.00%
| sion     | -0.000 | 100.00%
| limit    | -0.000 | 99.99%
| (        | -0.006 | 99.39%
| 10       | -0.339 | 71.26%


In [108]:
len(generated_tokens[0]), len(transition_scores[0])

(29, 29)

In [109]:
text = do_predict(["Write a hello world program in Python", "What is the fourth prime number?"], model, tokenizer, n=3, logprobs = False)


In [110]:
text

['ok=0 while y=="" and y!=0 while y!= 0: y+=',
 'i = 1 while i  len(stdin) : i += 1 if i!',
 'vf = lambda x, y: [x,y]; vf = vf(); ',
 'ten',
 'ten',
 '14']

In [31]:
input_text = "For Halloween Debby and her sister combined the candy they received. Debby had 32 pieces of candy while her sister had 42. If they ate 35 pieces the first night, how many pieces do they have left?"
preprompt_1 = ""
postprompt_1 = ""

preprompt_2 = ""
postprompt_2 = "Let's think step by step."

input_ids_1 = tokenizer(preprompt_1 + input_text + postprompt_1, return_tensors="pt").input_ids.to(dev)
input_ids_2 = tokenizer(preprompt_2 + input_text + postprompt_2, return_tensors="pt").input_ids.to(dev)